# 3.0 Modelado y Predicción de Demanda

En este notebook, entrenaremos varios de Machine Learning para predecir la demanda de taxis. Utilizaremos las características creadas en el notebook anterior y evaluaremos el rendimiento de los modelos con la métrica RMSE.

In [1]:
from pathlib import Path
import sys

current_working_directory = Path.cwd()
project_root = current_working_directory.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [2]:
# Bibliotecas necesarias
import pandas as pd
import os

from src.models.train_model import train_lr_model, train_dt_model, train_rf_model, train_lgbt_model, train_cb_model, train_xgb_model
from src.utils.helpers import print_rmse, results_to_parquet

### Carga de Datos con Características

Cargamos el DataFrame que ya procesado.

In [3]:
features_data_path = '../data/processed/features_taxi_data.parquet'

if not os.path.exists(features_data_path):
    print(f"Error: No se encontró el archivo de características en {features_data_path}.")
    print("Por favor, ejecuta el notebook '2.0-ingenieria-caracteristicas-preparacion.ipynb' primero.")
    df_features = None
else:
    df_features = pd.read_parquet(features_data_path)
    print(f"Datos cargados desde: {features_data_path}")
    print(df_features.head())

Datos cargados desde: ../data/processed/features_taxi_data.parquet
                     num_orders  year  month  day  dayofweek  hour  lag_1  \
datetime                                                                    
2018-03-01 00:00:00         124  2018      3    1          3     0    NaN   
2018-03-01 01:00:00          85  2018      3    1          3     1  124.0   
2018-03-01 02:00:00          71  2018      3    1          3     2   85.0   
2018-03-01 03:00:00          66  2018      3    1          3     3   71.0   
2018-03-01 04:00:00          43  2018      3    1          3     4   66.0   

                     lag_2  lag_3  lag_4  ...  lag_16  lag_17  lag_18  lag_19  \
datetime                                  ...                                   
2018-03-01 00:00:00    NaN    NaN    NaN  ...     NaN     NaN     NaN     NaN   
2018-03-01 01:00:00    NaN    NaN    NaN  ...     NaN     NaN     NaN     NaN   
2018-03-01 02:00:00  124.0    NaN    NaN  ...     NaN     NaN     NaN

### Entrenamiento del Modelos

Entrenaremos los modelos de `Regresión lineal`,`Arbol de decisiones`,`Bosque aleatorio`,`LightBTM`,`CatBoost` y `XGBoost` a traves de sus propias funciones, las cuales realizarán una busqueda de sus mejores hiperparametros. Los modelos entrenados serán guardados.

In [4]:
# Regresión lineal 
if df_features is not None:
    TARGET_COLUMN = 'num_orders'
    MODEL_SAVE_PATH = '../models/lr_taxi_demand_model.pkl'
    RESULT_SAVE_PATH ='../data/processed/test_predictions.parquet'
    TEST_SIZE = 0.1
    STATE = 31415

    # Entrenamiento del modelo
    best_model, features_test, target_test, predictions, times = train_lr_model(
        df_features.copy(),
        TARGET_COLUMN,
        test_size=TEST_SIZE,
        state=STATE,
        model_save_path=MODEL_SAVE_PATH
    )

    print('Evaluación del modelo final en el conjunto de pruebas:')
    rmse_test = print_rmse(target_test, predictions['test'])

    print(f"El RMSE en el conjunto de prueba es: {round(rmse_test, 2)}")
    if rmse_test <= 48:
        print("¡El modelo cumple el requisito de RMSE (<= 48)!")
    else:
        print("El modelo NO cumple el requisito de RMSE (debe ser <= 48). Se necesita optimización adicional.")

    predictions_test_df = pd.DataFrame({
        'real': target_test,
        'predicted_lr': predictions['test']
    })

    results_to_parquet(predictions_test_df, RESULT_SAVE_PATH)

else:
    print("No se pudo entrenar el modelo porque los datos de características no se cargaron.")


Tamaños conjunto de entrenamiento: (3952, 30)
Tamaños conjunto de prueba: (440, 30)

Entrenando el modelo...
Tiempo de entrenamiento: 0.0106 s
Tiempo de predicción: 0.001 s
Modelo entrenado guardado en: ../models/lr_taxi_demand_model.pkl
Evaluación del modelo final en el conjunto de pruebas:
El RMSE calculado es: 45.83
El RMSE en el conjunto de prueba es: 45.83
¡El modelo cumple el requisito de RMSE (<= 48)!
Archivo actualizado en: ../data/processed/test_predictions.parquet


In [5]:
times

{'train': 0.010565996170043945, 'predict': 0.0010101795196533203}

In [6]:
# Arbol de decisiones 
if df_features is not None:
    TARGET_COLUMN = 'num_orders'
    MODEL_SAVE_PATH = '../models/dt_taxi_demand_model.pkl'
    RESULT_SAVE_PATH ='../data/processed/test_predictions.parquet'
    TEST_SIZE = 0.1
    STATE = 31415

    # Entrenamiento del modelo
    best_model, features_test, target_test, predictions = train_dt_model(
        df_features.copy(),
        TARGET_COLUMN,
        test_size=TEST_SIZE,
        state=STATE,
        model_save_path=MODEL_SAVE_PATH
    )

    print('Evaluación del modelo final en el conjunto de pruebas:')
    rmse_test = print_rmse(target_test, predictions['test'])

    print(f"El RMSE en el conjunto de prueba es: {round(rmse_test, 2)}")
    if rmse_test <= 48:
        print("¡El modelo cumple el requisito de RMSE (<= 48)!")
    else:
        print("El modelo NO cumple el requisito de RMSE (debe ser <= 48). Se necesita optimización adicional.")

    predictions_test_df = pd.DataFrame({
        'real': target_test,
        'predicted_dt': predictions['test']
    })
    
    results_to_parquet(predictions_test_df, RESULT_SAVE_PATH)

else:
    print("No se pudo entrenar el modelo porque los datos de características no se cargaron.")


Tamaños conjunto de entrenamiento: (3952, 30)
Tamaños conjunto de prueba: (440, 30)

Buscando los mejores hiperparámetros para Arbol de decisiones...
Mejores hiperparámetros encontrados:
{'max_depth': 4}

Entrenando el modelo final con los mejores hiperparámetros...

Realizando predicciones...
Tiempo de entrenamiento: 0.0198 s
Tiempo de predicción: 0.001 s
Modelo entrenado guardado en: ../models/dt_taxi_demand_model.pkl
Evaluación del modelo final en el conjunto de pruebas:
El RMSE calculado es: 54.61
El RMSE en el conjunto de prueba es: 54.61
El modelo NO cumple el requisito de RMSE (debe ser <= 48). Se necesita optimización adicional.
Predicciones del conjunto de prueba guardadas en:  ../data/processed/test_predictions.parquet


In [7]:
# Bosque aleatorio
if df_features is not None:
    TARGET_COLUMN = 'num_orders'
    MODEL_SAVE_PATH = '../models/rf_taxi_demand_model.pkl'
    RESULT_SAVE_PATH ='../data/processed/test_predictions.parquet'
    TEST_SIZE = 0.1
    STATE = 31415

    # Entrenamiento del modelo
    best_model, features_test, target_test, predictions = train_rf_model(
        df_features.copy(),
        TARGET_COLUMN,
        test_size=TEST_SIZE,
        state=STATE,
        model_save_path=MODEL_SAVE_PATH
    )

    print('Evaluación del modelo final en el conjunto de pruebas:')
    rmse_test = print_rmse(target_test, predictions['test'])

    print(f"El RMSE en el conjunto de prueba es: {round(rmse_test, 2)}")
    if rmse_test <= 48:
        print("¡El modelo cumple el requisito de RMSE (<= 48)!")
    else:
        print("El modelo NO cumple el requisito de RMSE (debe ser <= 48). Se necesita optimización adicional.")

    predictions_test_df = pd.DataFrame({
        'real': target_test,
        'predicted_rf': predictions['test']
    })
    
    results_to_parquet(predictions_test_df, RESULT_SAVE_PATH)

else:
    print("No se pudo entrenar el modelo porque los datos de características no se cargaron.")


Tamaños conjunto de entrenamiento: (3952, 30)
Tamaños conjunto de prueba: (440, 30)

Buscando los mejores hiperparámetros para Bosque aleatorio...
Mejores hiperparámetros encontrados:
{'max_depth': 12, 'n_estimators': 40}

Entrenando el modelo final con los mejores hiperparámetros...

Realizando predicciones...
Tiempo de entrenamiento: 1.7873 s
Tiempo de predicción: 0.0032 s
Modelo entrenado guardado en: ../models/rf_taxi_demand_model.pkl
Evaluación del modelo final en el conjunto de pruebas:
El RMSE calculado es: 44.78
El RMSE en el conjunto de prueba es: 44.78
¡El modelo cumple el requisito de RMSE (<= 48)!
Predicciones del conjunto de prueba guardadas en:  ../data/processed/test_predictions.parquet


In [8]:
# LightGBT
if df_features is not None:
    TARGET_COLUMN = 'num_orders'
    MODEL_SAVE_PATH = '../models/lgbt_taxi_demand_model.pkl'
    RESULT_SAVE_PATH ='../data/processed/test_predictions.parquet'
    TEST_SIZE = 0.1
    STATE = 31415

    # Entrenamiento del modelo
    best_model, features_test, target_test, predictions = train_lgbt_model(
        df_features.copy(),
        TARGET_COLUMN,
        test_size=TEST_SIZE,
        state=STATE,
        model_save_path=MODEL_SAVE_PATH
    )

    print('Evaluación del modelo final en el conjunto de pruebas:')
    rmse_test = print_rmse(target_test, predictions['test'])

    print(f"El RMSE en el conjunto de prueba es: {round(rmse_test, 2)}")
    if rmse_test <= 48:
        print("¡El modelo cumple el requisito de RMSE (<= 48)!")
    else:
        print("El modelo NO cumple el requisito de RMSE (debe ser <= 48). Se necesita optimización adicional.")

    predictions_test_df = pd.DataFrame({
        'real': target_test,
        'predicted_lgbt': predictions['test']
    })
    
    results_to_parquet(predictions_test_df, RESULT_SAVE_PATH)

else:
    print("No se pudo entrenar el modelo porque los datos de características no se cargaron.")

Tamaños conjunto de entrenamiento: (3952, 30)
Tamaños conjunto de prueba: (440, 30)

Buscando los mejores hiperparámetros para LightGBM...
Mejores hiperparámetros encontrados:
{'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 100, 'num_leaves': 10}

Entrenando el modelo final con los mejores hiperparámetros...

Realizando predicciones...
Tiempo de entrenamiento: 0.0459 s
Tiempo de predicción: 0.001 s
Modelo entrenado guardado en: ../models/lgbt_taxi_demand_model.pkl
Evaluación del modelo final en el conjunto de pruebas:
El RMSE calculado es: 41.21
El RMSE en el conjunto de prueba es: 41.21
¡El modelo cumple el requisito de RMSE (<= 48)!
Predicciones del conjunto de prueba guardadas en:  ../data/processed/test_predictions.parquet


In [9]:
# CatBoost
if df_features is not None:
    TARGET_COLUMN = 'num_orders'
    MODEL_SAVE_PATH = '../models/cb_taxi_demand_model.pkl'
    RESULT_SAVE_PATH ='../data/processed/test_predictions.parquet'
    TEST_SIZE = 0.1
    STATE = 31415

    # Entrenamiento del modelo
    best_model, features_test, target_test, predictions = train_cb_model(
        df_features.copy(),
        TARGET_COLUMN,
        test_size=TEST_SIZE,
        state=STATE,
        model_save_path=MODEL_SAVE_PATH
    )

    print('Evaluación del modelo final en el conjunto de pruebas:')
    rmse_test = print_rmse(target_test, predictions['test'])

    print(f"El RMSE en el conjunto de prueba es: {round(rmse_test, 2)}")
    if rmse_test <= 48:
        print("¡El modelo cumple el requisito de RMSE (<= 48)!")
    else:
        print("El modelo NO cumple el requisito de RMSE (debe ser <= 48). Se necesita optimización adicional.")

    predictions_test_df = pd.DataFrame({
        'real': target_test,
        'predicted_cb': predictions['test']
    })
    
    results_to_parquet(predictions_test_df, RESULT_SAVE_PATH)

else:
    print("No se pudo entrenar el modelo porque los datos de características no se cargaron.")

Tamaños conjunto de entrenamiento: (3952, 30)
Tamaños conjunto de prueba: (440, 30)

Buscando los mejores hiperparámetros para CatBoost...
Mejores hiperparámetros encontrados:
{'depth': 5, 'iterations': 500, 'learning_rate': 0.1}

Entrenando el modelo final con los mejores hiperparámetros...

Realizando predicciones...
Tiempo de entrenamiento: 0.9002 s
Tiempo de predicción: 0.0036 s
Modelo entrenado guardado en: ../models/cb_taxi_demand_model.pkl
Evaluación del modelo final en el conjunto de pruebas:
El RMSE calculado es: 41.24
El RMSE en el conjunto de prueba es: 41.24
¡El modelo cumple el requisito de RMSE (<= 48)!
Predicciones del conjunto de prueba guardadas en:  ../data/processed/test_predictions.parquet


In [10]:
# XGBoost
if df_features is not None:
    TARGET_COLUMN = 'num_orders'
    MODEL_SAVE_PATH = '../models/xgb_taxi_demand_model.pkl'
    RESULT_SAVE_PATH ='../data/processed/test_predictions.parquet'
    TEST_SIZE = 0.1
    STATE = 31415

    # Entrenamiento del modelo
    best_model, features_test, target_test, predictions = train_xgb_model(
        df_features.copy(),
        TARGET_COLUMN,
        test_size=TEST_SIZE,
        state=STATE,
        model_save_path=MODEL_SAVE_PATH
    )

    print('Evaluación del modelo final en el conjunto de pruebas:')
    rmse_test = print_rmse(target_test, predictions['test'])

    print(f"El RMSE en el conjunto de prueba es: {round(rmse_test, 2)}")
    if rmse_test <= 48:
        print("¡El modelo cumple el requisito de RMSE (<= 48)!")
    else:
        print("El modelo NO cumple el requisito de RMSE (debe ser <= 48). Se necesita optimización adicional.")

    predictions_test_df = pd.DataFrame({
        'real': target_test,
        'predicted_xgb': predictions['test']
    })
    
    results_to_parquet(predictions_test_df, RESULT_SAVE_PATH)

else:
    print("No se pudo entrenar el modelo porque los datos de características no se cargaron.")

Tamaños conjunto de entrenamiento: (3952, 30)
Tamaños conjunto de prueba: (440, 30)

Buscando los mejores hiperparámetros para XGBoost...
Mejores hiperparámetros encontrados:
{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}

Entrenando el modelo final con los mejores hiperparámetros...

Realizando predicciones...
Tiempo de entrenamiento: 0.11 s
Tiempo de predicción: 0.003 s
Modelo entrenado guardado en: ../models/xgb_taxi_demand_model.pkl
Evaluación del modelo final en el conjunto de pruebas:
El RMSE calculado es: 41.86
El RMSE en el conjunto de prueba es: 41.86
¡El modelo cumple el requisito de RMSE (<= 48)!
Predicciones del conjunto de prueba guardadas en:  ../data/processed/test_predictions.parquet
